In [1]:

# Import common dependencies
import pandas as pd  # noqa
import numpy as np
import matplotlib  # noqa
import matplotlib.pyplot as plt
import datetime  # noqa
import PIL  # noqa
import glob  # noqa
import pickle  # noqa
from pathlib import Path  # noqa
from scipy import misc  # noqa
import sys
import tensorflow as tf
import pdb
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
TRADE_COST_FRAC = .003
EPSILON = 1e-10
ADV_MULT = 1e-3

In [2]:
uni_tokens = set()
uni_commands = set()
uni_actions = set()
fname = 'tasks_with_length_tags.txt'
with open(fname) as f:
    content = f.readlines()
content2 = [c.split(' ') for c in content]
# you may also want to remove whitespace characters like `\n` at the end of each line
commands = []
actions = []
content = [l.replace('\n', '') for l in content]
commands = [x.split(':::')[1].split(' ')[1:-1] for x in content]
actions = [x.split(':::')[2].split(' ')[1:-2] for x in content]
structures = [x.split(':::')[3].split(' ')[2:] for x in content]

structures = [[int(l) for l in program] for program in structures]
#actions = [[wd.replace('\n', '') for wd in res] for res in actions]

In [3]:
max_actions_per_subprogram = max([max([s for s in struct]) for struct in structures]) + 1
max_num_subprograms = max([len(s) for s in structures]) + 1
max_cmd_len = max([len(s) for s in commands]) + 1
max_act_len = max([len(a) for a in actions]) + 1
cmd_lengths_list = [len(s)+1 for s in commands]
cmd_lengths_np = np.array(cmd_lengths_list)
max_num_subprograms, max_cmd_len, max_act_len, max_actions_per_subprogram


(7, 10, 49, 9)

In [4]:
def build_fmap_invmap(unique, num_unique):
    fmap = dict(zip(unique, range(num_unique)))
    invmap = dict(zip(range(num_unique), unique))
    return fmap, invmap


In [5]:
for li in content2:
    for wd in li:
        uni_tokens.add(wd)
for li in commands:
    for wd in li:
        uni_commands.add(wd)
for li in actions:
    for wd in li:
        uni_actions.add(wd)
uni_commands.add('end_command')
uni_actions.add('end_subprogram')
uni_actions.add('end_action')
num_cmd = len(uni_commands)
num_act = len(uni_actions)
command_map, command_invmap = build_fmap_invmap(uni_commands, num_cmd)
action_map, action_invmap = build_fmap_invmap(uni_actions, num_act)

In [6]:


def dense_scaled(prev_layer, layer_size, name=None, reuse=False, scale=1.0):
    output = tf.layers.dense(prev_layer, layer_size, reuse=reuse) * scale
    return output


def dense_relu(dense_input, layer_size, scale=1.0):
    dense = dense_scaled(dense_input, layer_size, scale=scale)
    output = tf.nn.leaky_relu(dense)

    return output

def get_grad_norm(opt_fcn, loss):
    gvs = opt_fcn.compute_gradients(loss)
    grad_norm = tf.sqrt(tf.reduce_sum(
        [tf.reduce_sum(tf.square(grad)) for grad, var in gvs if grad is not None]))
    return grad_norm


def apply_clipped_optimizer(opt_fcn, loss, clip_norm=.1, clip_single=.03, clip_global_norm=False):
    gvs = opt_fcn.compute_gradients(loss)

    if clip_global_norm:
        gs, vs = zip(*[(g, v) for g, v in gvs if g is not None])
        capped_gs, grad_norm_total = tf.clip_by_global_norm([g for g in gs], clip_norm)
        capped_gvs = list(zip(capped_gs, vs))
    else:
        grad_norm_total = tf.sqrt(
            tf.reduce_sum([tf.reduce_sum(tf.square(grad)) for grad, var in gvs if grad is not None]))
        capped_gvs = [(tf.clip_by_value(grad, -1 * clip_single, clip_single), var)
                      for grad, var in gvs if grad is not None]
        capped_gvs = [(tf.clip_by_norm(grad, clip_norm), var) for grad, var in capped_gvs if grad is not None]

    optimizer = opt_fcn.apply_gradients(capped_gvs)

    return optimizer, grad_norm_total


def mlp(x, hidden_sizes, output_size=None, name='', reuse=False):
    prev_layer = x

    for idx, l in enumerate(hidden_sizes):
        dense = dense_scaled(prev_layer, l, name='mlp' + name + '_' + str(idx))
        prev_layer = tf.nn.leaky_relu(dense)

    output = prev_layer

    if output_size is not None:
        output = dense_scaled(prev_layer, output_size, name='mlp' + name + 'final')

    return output

def mlp_with_adversaries(x, hidden_sizes, output_size=None, name='', reuse=False):
    prev_layer = x
    adv_phs = []
    for idx, l in enumerate(hidden_sizes):
        
        adversary = tf.placeholder_with_default(tf.zeros_like(prev_layer), prev_layer.shape)
        prev_layer = prev_layer + adversary
        adv_phs.append(adversary)
        
        dense = dense_scaled(prev_layer, l, name='mlp' + name + '_' + str(idx))
        prev_layer = tf.nn.leaky_relu(dense)

    output = prev_layer

    if output_size is not None:
        output = dense_scaled(prev_layer, output_size, name='mlp' + name + 'final')

    return output, adv_phs



In [7]:

commands_ind = [[command_map[c] for c in cmd] + [0] * (max_cmd_len - len(cmd)) for cmd in commands]
actions_ind = [[action_map[a] for a in act] + [0] * (max_act_len - len(act)) for act in actions]
cmd_np = np.array(commands_ind)
actions_structured = []
mask_structured = []
for row in range(len(structures)):
    mask_row = []
    action_row = []
    act = actions_ind[row]
    struct = structures[row]
    start = 0
    for step in struct:
        end = start + step
        a = act[start:end]
        padding = max_actions_per_subprogram - step - 1
        action_row.append(a + [action_map['end_action']] + [0] * padding)
        start = end
    actions_structured.append(
        action_row + [[action_map['end_subprogram']] + [0] * (max_actions_per_subprogram - 1)] +
        [[0] * max_actions_per_subprogram] * (max_num_subprograms - len(struct) - 1)
    )
act_np = np.array(actions_structured)
struct_padded = [[sa + 1 for sa in s] + [1] + [0] * (max_num_subprograms - len(s) - 1) for s in structures]
struct_np = np.array(struct_padded)

mask_list = [[np.concatenate((np.ones(st), np.zeros(max_actions_per_subprogram - st)), 0) 
              for st in s] for s in struct_np]
mask_np = np.array(mask_list)

In [8]:
tf.reset_default_graph()
default_sizes = 128
size_emb = 64
num_layers_encoder = 6
hidden_filters = 256
num_layers_subprogram = 3
hidden_filters_subprogram = 256
init_mag = 1e-3
cmd_mat = tf.Variable(init_mag*tf.random_normal([num_cmd, size_emb]))
act_mat = tf.Variable(init_mag*tf.random_normal([num_act, size_emb]))
act_st_emb = tf.Variable(init_mag*tf.random_normal([size_emb]))
global_bs = None
global_time_len = 7
action_lengths = None
max_num_actions= None
# global_bs = 8
global_time_len = 7
max_num_actions = 9
output_keep_prob = tf.placeholder_with_default(1.0, ())
state_keep_prob = tf.placeholder_with_default(1.0, ())
cmd_ind = tf.placeholder(tf.int32, shape=(global_bs, 10,))
act_ind = tf.placeholder(tf.int32, shape=(global_bs, global_time_len, 9))
mask_ph = tf.placeholder(tf.float32, shape=(global_bs, global_time_len, 9))
cmd_lengths = tf.placeholder(tf.int32, shape=(global_bs,))
act_lengths = tf.placeholder(tf.int32, shape=(global_bs, 7))
learning_rate = tf.placeholder(tf.float32, shape = (None))

cmd_emb = tf.nn.embedding_lookup(cmd_mat, cmd_ind)
act_emb = tf.nn.embedding_lookup(act_mat, act_ind)
tf_bs = tf.shape(act_ind)[0]
act_st_emb_expanded = tf.tile(tf.reshape(
    act_st_emb, [1, 1, 1, size_emb]), [tf_bs, global_time_len, 1, 1])
act_emb_with_st = tf.concat((act_st_emb_expanded, act_emb), 2)

first_cell_encoder = [tf.nn.rnn_cell.LSTMCell(
    hidden_filters, forget_bias=1., name = 'layer1_'+d) for d in ['f', 'b']]
hidden_cells_encoder = [[tf.nn.rnn_cell.LSTMCell(
    hidden_filters,forget_bias=1., name = 'layer' + str(lidx) + '_' + d)  for d in ['f', 'b']]
                        for lidx in range(num_layers_encoder - 1)]
hidden_cells_encoder = [[tf.nn.rnn_cell.DropoutWrapper(cell,
    output_keep_prob=output_keep_prob, state_keep_prob=state_keep_prob,
    variational_recurrent=True, dtype=tf.float32) for cell in cells] 
                        for cells in hidden_cells_encoder[:-1]] + [hidden_cells_encoder[-1]]
cells_encoder = [first_cell_encoder] + hidden_cells_encoder
c1, c2 = zip(*cells_encoder)
cells_encoder = [c1, c2]
def encode(x, num_layers, cells, initial_states, lengths, name='',):
    prev_layer = x
    shortcut = x
    hiddenlayers = []
    returncells = []
    cell_fw, cell_bw = cells
    bs = tf.shape(x)[0]
    for idx in range(num_layers):
        prev_layer, c = tf.nn.bidirectional_dynamic_rnn(
                cell_fw = cell_fw[idx],
                cell_bw = cell_bw[idx],
                inputs = prev_layer,
                sequence_length=lengths,
                initial_state_fw=None,
                initial_state_bw=None,
                dtype=tf.float32,
                scope='encoder'+str(idx)
            )
        prev_layer = tf.concat(prev_layer, 2)
        prev_layer = tf.nn.leaky_relu(prev_layer)
        returncells.append(c)
        hiddenlayers.append(prev_layer)
        if idx == num_layers - 1:
            #pdb.set_trace()
            output = tf.gather_nd(
                        prev_layer,
                        tf.stack([tf.range(bs), lengths], 1),
                        name=None
                    )
            return prev_layer, returncells, hiddenlayers, output
        prev_layer = tf.concat((prev_layer, shortcut), 2)
encoding_last_layer, encoding_final_cells, encoding_hidden_layers, encoding_last_timestep = encode(
    cmd_emb, num_layers_encoder, cells_encoder,None, lengths = cmd_lengths, name = 'encoder')
# encoding_last_timestep = encoding_last_layer[:,cmd_lengths, :]
hidden_filters_encoder = encoding_last_timestep.shape[-1].value
first_cell_subprogram = tf.nn.rnn_cell.LSTMCell(
    hidden_filters_subprogram, forget_bias=1., name = 'subpogramlayer1_')
hidden_cells_subprogram = [tf.nn.rnn_cell.LSTMCell(
    hidden_filters_subprogram,forget_bias=1., name = 'subpogramlayer' + str(lidx))
                        for lidx in range(num_layers_subprogram - 1)]

cells_subprogram_rnn = [first_cell_subprogram] + hidden_cells_subprogram

attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(
    num_units=hidden_filters_encoder, memory=encoding_last_layer,
    memory_sequence_length=cmd_lengths)
attention_mechanism = tf.contrib.seq2seq.LuongAttention(
    num_units=hidden_filters_encoder//2, memory=encoding_last_layer,
    memory_sequence_length=cmd_lengths)
cells_subprogram = [
    tf.contrib.seq2seq.AttentionWrapper(
        cell, attention_mechanism, attention_layer_size = hidden_filters_subprogram) 
    for cell in cells_subprogram_rnn]

def subprogram(x, num_layers, cells, initial_states, lengths, reuse, name='',):
    prev_layer = x
    shortcut = x
    hiddenlayers = []
    returncells = []
    bs = tf.shape(x)[0]
    for idx in range(num_layers):
        print(idx)
        if idx == 0:
            num_past_units = hidden_filters
        else:
            num_past_units = hidden_filters_subprogram
        with tf.variable_scope(name + 'subprogram' + str(idx), reuse=reuse):
#             self_attention_mechanism = tf.contrib.seq2seq.LuongAttention(
#                 num_units=num_past_units, memory=prev_layer,
#                 memory_sequence_length=tf.expand_dims(tf.range(10), 0))
#             cell_with_selfattention = tf.contrib.seq2seq.AttentionWrapper(
#                     cells[idx], self_attention_mechanism, attention_layer_size = num_past_units)

            prev_layer, c = tf.nn.dynamic_rnn(
                    cell = cells[idx],
                    inputs = prev_layer,
                    sequence_length=lengths,
                    initial_state = None,
                    dtype=tf.float32,
                )
            prev_layer = tf.concat(prev_layer, 2)
            prev_layer = tf.nn.leaky_relu(prev_layer)
            returncells.append(c)
            hiddenlayers.append(prev_layer)
            if idx == num_layers - 1:
                output = tf.gather_nd(
                            prev_layer,
                            tf.stack([tf.range(bs), lengths], 1),
                            name=None
                        )
                return prev_layer, returncells, hiddenlayers, output
            prev_layer = tf.concat((prev_layer, shortcut), 2)
encodings = [encoding_last_timestep]
last_encoding = encoding_last_timestep
initial_cmb_encoding = last_encoding
loss = 0
action_probabilities_presoftmax = []
for sub_idx in range(max_num_subprograms): 
    from_last_layer = tf.tile(tf.expand_dims(tf.concat((
        last_encoding, initial_cmb_encoding), 1), 1), [1, max_num_actions + 1, 1])
    autoregressive = act_emb_with_st[:,sub_idx, :, :]
    x_input = tf.concat((from_last_layer, autoregressive), -1)
    subprogram_last_layer, _, subprogram_hidden_layers, subprogram_output = subprogram(
        x_input, num_layers_subprogram, cells_subprogram,None, 
        lengths = act_lengths[:, sub_idx], reuse = (sub_idx > 0), name = 'subprogram')
    action_prob_flat = mlp(
        tf.reshape(subprogram_last_layer, [-1, hidden_filters_subprogram]),
        [], output_size = num_act, name = 'action_choice_mlp', reuse = (sub_idx > 0))
    action_prob_expanded = tf.reshape(action_prob_flat, [-1, max_num_actions + 1, num_act])
    action_probabilities_layer = tf.nn.softmax(action_prob_expanded, axis=-1)
    action_probabilities_presoftmax.append(action_prob_expanded)
    delta = mlp(
        subprogram_output, [512,], output_size = hidden_filters_encoder, name = 'global_transform',
        reuse = (sub_idx > 0)
    )
    last_encoding = last_encoding + delta
    encodings.append(last_encoding)
act_presoftmax = tf.stack(action_probabilities_presoftmax, 1)[:, :, :-1, :]
#batch, subprogram, timestep, action_selection
logprobabilities = tf.nn.log_softmax(act_presoftmax, -1)
act_presoftmax_flat = tf.reshape(act_presoftmax, [-1, 9, num_act])
mask_ph_flat = tf.reshape(mask_ph, [-1, max_actions_per_subprogram])
act_ind_flat = tf.reshape(act_ind, [-1, max_actions_per_subprogram])
ppl_loss = tf.contrib.seq2seq.sequence_loss(
    logits = act_presoftmax_flat,
    targets = act_ind_flat,
    weights = mask_ph_flat,
    average_across_timesteps=False,
    average_across_batch=False,
    softmax_loss_function=None,
    name=None
)
ppl_loss_avg = tf.reduce_mean(tf.pow(ppl_loss, 2.0)) * 10000 # + tf.reduce_mean(tf.pow(ppl_loss, 1.0)) * 100

tfvars = tf.trainable_variables()
weight_norm = tf.reduce_mean([tf.reduce_sum(tf.square(var)) for var in tfvars])*1e-3

action_taken = tf.argmax(act_presoftmax, -1, output_type=tf.int32)
correct_mat = tf.cast(tf.equal(action_taken, act_ind), tf.float32) * mask_ph
correct_percent = tf.reduce_sum(correct_mat, [1, 2])/tf.reduce_sum(mask_ph, [1, 2])
percent_correct = tf.reduce_mean(correct_percent)
percent_fully_correct = tf.reduce_mean(tf.cast(tf.equal(correct_percent, 1), tf.float32))

loss = ppl_loss_avg + weight_norm

opt_fcn = tf.train.AdamOptimizer(learning_rate=learning_rate)
#opt_fcn = tf.train.MomentumOptimizer(learning_rate=learning_rate, use_nesterov=True, momentum=.8)
optimizer, grad_norm_total = apply_clipped_optimizer(opt_fcn, loss)
sess = tf.Session()
sess.run(tf.global_variables_initializer())

0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
0
1
2
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [9]:
hidden_filters

256

In [10]:
encoding_last_layer

<tf.Tensor 'LeakyRelu_5/Maximum:0' shape=(?, 10, 512) dtype=float32>

In [11]:
np.random.seed(0)
trn_percent = .1
num_samples = mask_np.shape[0]
ordered_samples = np.arange(num_samples)
np.random.shuffle(ordered_samples)
trn_samples = ordered_samples[:int(np.ceil(num_samples*trn_percent))]
val_samples_all = ordered_samples[int(np.ceil(num_samples*trn_percent)):]
val_samples = val_samples_all
trn_samples.shape, val_samples.shape

((2091,), (18819,))

In [12]:
eval_itr = -1
bs = trn_samples.shape[0]
for itr in range(100000):
    if itr == 0:
        samples = np.random.choice(trn_samples, size = bs, replace = False)
        trn_feed_dict = {
            cmd_ind : cmd_np[samples],
            act_ind : act_np[samples],
            mask_ph : mask_np[samples],
            act_lengths : np.clip(struct_np[samples], a_min = 1, a_max = None),
            cmd_lengths : cmd_lengths_np[samples],
        }
        
    trn_feed_dict[learning_rate] = .02 / (np.power(itr + 10, .6))
    _, trn_loss, acc_trn_single, acc_trn = sess.run(
        [optimizer, loss, percent_correct, percent_fully_correct], trn_feed_dict)
    if itr == 0:
        trn_loss_avg = trn_loss
        acc_trn_avg = acc_trn
        acc_trn_single_avg = acc_trn_single
    else:
        trn_loss_avg = trn_loss_avg * .9 + trn_loss * .1
        acc_trn_avg = acc_trn_avg * .9 + acc_trn * .1
        acc_trn_single_avg = acc_trn_single_avg * .9 + acc_trn_single * .1
    if itr % 10 == 0 and itr > 0:
        # val_samples = np.random.choice(val_samples_all, size = bs, replace = False)
        eval_itr += 1
        val_feed_dict = {
            cmd_ind : cmd_np[val_samples],
            act_ind : act_np[val_samples],
            mask_ph : mask_np[val_samples],
            act_lengths : np.clip(struct_np[val_samples], a_min = 1, a_max = None),
            cmd_lengths : cmd_lengths_np[val_samples]
        }
        val_loss, acc_val = sess.run([loss, percent_fully_correct], val_feed_dict)
        if eval_itr == 0:
            val_loss_avg = val_loss
            acc_val_avg = acc_val
        else:
            val_loss_avg = val_loss_avg * .9 + val_loss * .1
            acc_val_avg = acc_val_avg * .9 + acc_val * .1
        print('itr:', itr, 'trn_loss', trn_loss_avg, 'val_loss', val_loss_avg)
        print('itr:', itr, 'trn_acc', acc_trn_avg, 
              'trn_single_acc', acc_trn_single_avg, 'val_acc', acc_val_avg)

itr: 10 trn_loss 17842.486279513694 val_loss 10478.603
itr: 10 trn_acc 0.0 trn_single_acc 0.223904863236759 val_acc 0.0
itr: 20 trn_loss 12562.267303504317 val_loss 10328.4494140625
itr: 20 trn_acc 2.5415638595481877e-05 trn_single_acc 0.2987364084533328 val_acc 0.0
itr: 30 trn_loss 9770.22325726162 val_loss 10014.775224609375
itr: 30 trn_acc 8.861885219617975e-06 trn_single_acc 0.3772129617746354 val_acc 0.0
itr: 40 trn_loss 7369.543157616672 val_loss 9576.405221679688
itr: 40 trn_acc 3.0899483147216424e-06 trn_single_acc 0.4661824691233861 val_acc 5.313778456184082e-06
itr: 50 trn_loss 5645.404576216715 val_loss 9039.316066699219
itr: 50 trn_acc 0.0001445494221323052 trn_single_acc 0.5385045825109004 val_acc 2.0723737070511563e-05
itr: 60 trn_loss 4273.4613724956525 val_loss 8461.374352607421
itr: 60 trn_acc 0.0018454128762757458 trn_single_acc 0.602307728005473 val_acc 0.00023651628188599717
itr: 70 trn_loss 3382.825179534208 val_loss 7896.342337268555
itr: 70 trn_acc 0.003291063973

itr: 520 trn_loss 192.76022831952253 val_loss 1397.558130605009
itr: 520 trn_acc 0.5165091701601002 trn_single_acc 0.9556212612342931 val_acc 0.1224612991579497
itr: 530 trn_loss 175.8878012091269 val_loss 1417.206211587477
itr: 530 trn_acc 0.5560367989432191 trn_single_acc 0.9597325077568042 val_acc 0.12956263726068654
itr: 540 trn_loss 153.98558105009576 val_loss 1449.2886788076355
itr: 540 trn_acc 0.6265486634589126 trn_single_acc 0.9659706315057298 val_acc 0.13619827573102522
itr: 550 trn_loss 138.04856278515547 val_loss 1489.0020594620282
itr: 550 trn_acc 0.6725685928354713 trn_single_acc 0.9703914426391411 val_acc 0.14489631811918444
itr: 560 trn_loss 137.61864381290877 val_loss 1553.768040039263
itr: 560 trn_acc 0.6719567277804247 trn_single_acc 0.9707456391523831 val_acc 0.150317415238693
itr: 570 trn_loss 122.86004636875568 val_loss 1600.324207226743
itr: 570 trn_acc 0.7098884777994262 trn_single_acc 0.9741369586109081 val_acc 0.15993629214877603
itr: 580 trn_loss 106.30364411

itr: 1030 trn_loss 7.634261272569635 val_loss 3978.6748234301945
itr: 1030 trn_acc 0.9933843918312726 trn_single_acc 0.9996238563123856 val_acc 0.42226885617014914
itr: 1040 trn_loss 4.758945341535878 val_loss 4034.6469895246755
itr: 1040 trn_acc 0.9956712714975209 trn_single_acc 0.9997425172716221 val_acc 0.4241782146117432
itr: 1050 trn_loss 4.005313356292805 val_loss 4028.5887602987705
itr: 1050 trn_acc 0.9970711888669681 trn_single_acc 0.9998305668430831 val_acc 0.42673621631455516
itr: 1060 trn_loss 4.272830446505382 val_loss 4049.9251967688933
itr: 1060 trn_acc 0.9968818978434049 trn_single_acc 0.9998166610842872 val_acc 0.42931473305933565
itr: 1070 trn_loss 3.5674727527601093 val_loss 4079.2070423263795
itr: 1070 trn_acc 0.997392369146866 trn_single_acc 0.9998509471742112 val_acc 0.43135908291738834
itr: 1080 trn_loss 5.557359464783332 val_loss 4093.1529396562414
itr: 1080 trn_acc 0.9939091410676221 trn_single_acc 0.9995969492313996 val_acc 0.4339854363739635
itr: 1090 trn_loss

itr: 1540 trn_loss 1.72155540040436 val_loss 4683.630968418498
itr: 1540 trn_acc 0.998376697557386 trn_single_acc 0.9999016673881006 val_acc 0.5139646152554598
itr: 1550 trn_loss 1.1752904792770664 val_loss 4666.923584467274
itr: 1550 trn_acc 0.9989908868036375 trn_single_acc 0.9999442946207758 val_acc 0.5137079614310247
itr: 1560 trn_loss 0.9761518586414953 val_loss 4671.918774848671
itr: 1560 trn_acc 0.9994233681997399 trn_single_acc 0.9999716860593435 val_acc 0.5165217630185588
itr: 1570 trn_loss 0.9161791419100075 val_loss 4650.054485254429
itr: 1570 trn_acc 0.9993901285337345 trn_single_acc 0.9999734226871878 val_acc 0.5184165339575324
itr: 1580 trn_loss 1.5046742107792332 val_loss 4679.785950791485
itr: 1580 trn_acc 0.9984286192720344 trn_single_acc 0.9999081120556967 val_acc 0.519372585496853
itr: 1590 trn_loss 1.226609618838977 val_loss 4664.728400634212
itr: 1590 trn_acc 0.99888861032983 trn_single_acc 0.9999413956319044 val_acc 0.5209291366471799
itr: 1600 trn_loss 0.89330336

itr: 2050 trn_loss 0.3304426511406776 val_loss 7245.747801012443
itr: 2050 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.566597645038907
itr: 2060 trn_loss 0.32954156494177844 val_loss 7277.904319739324
itr: 2060 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.5665880720805028
itr: 2070 trn_loss 0.32864407944135743 val_loss 7308.998946359142
itr: 2070 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.5665847731522529
itr: 2080 trn_loss 0.3277501299277576 val_loss 7339.162137660727
itr: 2080 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.5665392940841742
itr: 2090 trn_loss 0.3268596800790149 val_loss 7368.460962957155
itr: 2090 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.56648773541474
itr: 2100 trn_loss 0.3259726635163952 val_loss 7396.893138145815
itr: 2100 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.5664360158779351
itr: 2110 trn_loss 0.32

...

itr: 97810 trn_loss 0.012021751338698403 val_loss 1.8446923924603962
itr: 97810 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.998362364603972
itr: 97820 trn_loss 0.012015675675652232 val_loss 1.8484663873719187
itr: 97820 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9983560880129596
itr: 97830 trn_loss 0.012009579344284763 val_loss 1.8522922316021035
itr: 97830 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9983451223467344
itr: 97840 trn_loss 0.012003462401688514 val_loss 1.856142030849425
itr: 97840 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9983352532471318
itr: 97850 trn_loss 0.011997325204327131 val_loss 1.8600578546771718
itr: 97850 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9983263710574894
itr: 97860 trn_loss 0.011991166668814656 val_loss 1.8640267229303105
itr: 97860 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.99831837708

itr: 98300 trn_loss 0.011699428296124033 val_loss 2.079104543579773
itr: 98300 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9979769019994547
itr: 98310 trn_loss 0.01169231788362436 val_loss 2.084670556354364
itr: 98310 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9979719724100897
itr: 98320 trn_loss 0.011685185788331613 val_loss 2.0903382653902534
itr: 98320 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9979675357796611
itr: 98330 trn_loss 0.011678031879051597 val_loss 2.0960642178276685
itr: 98330 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9979635428122755
itr: 98340 trn_loss 0.011670856628136677 val_loss 2.10178699011381
itr: 98340 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9979546383677789
itr: 98350 trn_loss 0.011663660071294778 val_loss 2.1074792198667724
itr: 98350 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.99794130763341

itr: 98800 trn_loss 0.011317281005284792 val_loss 2.4449035656429814
itr: 98800 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9975141916604695
itr: 98810 trn_loss 0.011309089610207745 val_loss 2.4565915166119474
itr: 98810 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9974970847903363
itr: 98820 trn_loss 0.011300876594612532 val_loss 2.468682184706307
itr: 98820 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9974763718729024
itr: 98830 trn_loss 0.011292642399423949 val_loss 2.4810297026248365
itr: 98830 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9974630469815259
itr: 98840 trn_loss 0.011284387250264602 val_loss 2.493672543824755
itr: 98840 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.997451054579287
itr: 98850 trn_loss 0.011276111656793592 val_loss 2.5065485126325875
itr: 98850 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.997440261417

itr: 99300 trn_loss 0.01088213408325643 val_loss 3.419104310559577
itr: 99300 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9968132433649202
itr: 99310 trn_loss 0.010872916706222514 val_loss 3.4471796745246768
itr: 99310 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.996818404557374
itr: 99320 trn_loss 0.01086368014084539 val_loss 3.4761522168683516
itr: 99320 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9968230496305825
itr: 99330 trn_loss 0.010854424860467227 val_loss 3.505990060100035
itr: 99330 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9968219194226208
itr: 99340 trn_loss 0.010845150450255156 val_loss 3.53602729314185
itr: 99340 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9968209022354552
itr: 99350 trn_loss 0.010835856746092738 val_loss 3.5661547130967692
itr: 99350 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9968146700326922

itr: 99800 trn_loss 0.010398910187140011 val_loss 5.245146249464032
itr: 99800 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9962470027425063
itr: 99810 trn_loss 0.01038880667886552 val_loss 5.2891205084897965
itr: 99810 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9962397121743715
itr: 99820 trn_loss 0.010378686656882604 val_loss 5.332403176955393
itr: 99820 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9962331506630501
itr: 99830 trn_loss 0.010368549799809848 val_loss 5.376417536078701
itr: 99830 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9962272453028609
itr: 99840 trn_loss 0.010358396722309926 val_loss 5.419381861572393
itr: 99840 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.9962166137443765
itr: 99850 trn_loss 0.010348227787772469 val_loss 5.462766436950799
itr: 99850 trn_acc 0.9999999999999994 trn_single_acc 0.9999999999999994 val_acc 0.996201734567891

In [13]:
correct_percent.shape, percent_fully_correct.shape

(TensorShape([Dimension(None)]), TensorShape([]))

In [14]:
10, None, 7, 9, None

(10, None, 7, 9, None)

In [15]:
cmd_np.shape, act_np.shape, mask_np.shape, struct_np.shape, cmd_lengths_np.shape

((20910, 10), (20910, 7, 9), (20910, 7, 9), (20910, 7), (20910,))

In [16]:
cmd_ind.shape, act_ind.shape, mask_ph.shape, act_lengths.shape, cmd_lengths.shape

(TensorShape([Dimension(None), Dimension(10)]),
 TensorShape([Dimension(None), Dimension(7), Dimension(9)]),
 TensorShape([Dimension(None), Dimension(7), Dimension(9)]),
 TensorShape([Dimension(None), Dimension(7)]),
 TensorShape([Dimension(None)]))

In [17]:
trn_samples.shape, val_samples.shape

((2091,), (18819,))

In [18]:

act_presoftmax = tf.stack(action_probabilities_presoftmax, 1)[:, :, :-1, :]
#batch, subprogram, timestep, action_selection
logprobabilities = tf.nn.log_softmax(act_presoftmax, -1)
act_presoftmax_flat = tf.reshape(act_presoftmax, [-1, 9, num_act])
mask_ph_flat = tf.reshape(mask_ph, [-1, max_actions_per_subprogram])
act_ind_flat = tf.reshape(act_ind, [-1, max_actions_per_subprogram])

In [19]:
act_presoftmax_flat = tf.reshape(act_presoftmax, [-1, 9, num_act])

In [20]:
max_actions_per_subprogram

9

In [21]:
sess.run(act_presoftmax, feed_dict).shape

NameError: name 'feed_dict' is not defined

In [ ]:
action_map

In [ ]:
print(*actions_ind[2])

In [ ]:
command_map

In [ ]:
action_map

In [ ]:
subprogram_output

In [ ]:
subprogram_last_layer[:,cmd_lengths,:]

In [ ]:
encoding_last_layer

In [ ]:
tf.gather(
    encoding_last_layer,
    [1,2],
    axis=1
)

In [ ]:
tf.gather_nd(
    encoding_last_layer,
    np.array([[0,1,2,3,4], [1,4,3,2,5]]).T,
    name=None
)

In [ ]:
cmd_lengths

In [ ]:
def generate_command(sub_cmd, num_repeat):
    return sub_cmd * num_repeat

In [ ]:
def process_command(cmd):
    

In [ ]:
uni_commands

In [ ]:
uni_actions

In [ ]:
uni_tokens

In [ ]:
df.shape